In [1]:
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00


In [2]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.ZoKFEyvmWz/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.yQOyCnhc3f/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.SeVm3DAsGH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [3]:
#!apt-get update
#!apt-get install chromium chromium-driver

In [4]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [5]:
import re

import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


import time

In [6]:
from selenium import webdriver

driver = web_driver()

driver.get('https://www.daraz.pk/catalog/?q=ear+buds&_keyori=ss&from=input&spm=a2a0e.searchlist.search.go.13d6f07cw4Mrha')

driver.maximize_window()

In [8]:
#next_btn = 'ant-pagination-next'
#driver.find_element(By.CLASS_NAME , next_btn).click()

In [10]:
product_dataframe = []

for i in range(1,10):
    time.sleep(10)
    product_box_class = "box--pRqdD"

    name_class = "title--wFj93"
    current_price_class = "currency--GVKjl"
    delivery_class = "location--eh0Ro"
    no_of_reviews_class = "rating__review--ygkUy"

    products = driver.find_elements(By.CLASS_NAME , product_box_class)

    for i in products:

        name = i.find_element(By.CLASS_NAME , name_class).text

        current_price = i.find_element(By.CLASS_NAME , current_price_class).text
        delivery = i.find_element(By.CLASS_NAME , delivery_class).text

        try:
            old_price = i.find_element(By.CLASS_NAME , 'origPrice--AJxRs').text
        except:
            old_price = "Not Available"

        try:
            discount = i.find_element(By.CLASS_NAME , 'discount--HADrg').text
        except:
            discount = "Discount not offered"

        try:
            no_of_reviews = i.find_element(By.CLASS_NAME , no_of_reviews_class).text

        except:
            no_of_reviews = 0

        product_dataframe.append([name,current_price, discount,old_price, delivery,no_of_reviews])
        # driver.find_element(By.CLASS_NAME , next_btn).click()
    #driver.find_element(By.XPATH , '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div/ul/li[9]/a').click()


In [11]:
import pandas as pd

df = pd.DataFrame(data = product_dataframe , columns = ["Name","Current Price" ,"Discout" , "Old Price" ,"Shipping Status",\
                                                        "No Of Reviews"])
df

,Name,Current Price,Discout,Old Price,Shipping Status,No Of Reviews
0,Air Pro TWS Earbuds with Charging Case i12 Pro...,"Rs. 1,349",-39%,"Rs. 2,199",Free Shipping,(2873)
1,"Audionic Airbud 430 ENC & Gaming mode, Water R...","Rs. 3,499",-47%,"Rs. 6,599",Free Shipping,(352)
2,Branded M10 Buds Wireless Buds Bluetooth EarBu...,"Rs. 1,390",-77%,"Rs. 6,000",Free Shipping,(552)
3,M10 Air buds Wireless Bluetooth EarBuds Stereo...,Rs. 999,-50%,"Rs. 1,999",Free Shipping,(4)
4,Timack® Newest air buds G11/ X16 pro Wireless ...,"Rs. 1,199",-60%,"Rs. 3,000",Free Shipping,(312)
...,...,...,...,...,...,...
355,Branded M10 Buds Wireless Bluetooth EarBuds St...,"Rs. 1,049",-74%,"Rs. 3,999",Pakistan,(1)
356,Air Pro TWS Earbuds with Charging Case i12 Pro...,Rs. 12,Discount not offered,Not Available,Free Shipping,(12)
357,Air buds/ Bluetooth Earphones / wireless earbu...,"Rs. 1,089",-78%,"Rs. 5,000",Free Shipping,(9)
358,Real_me_Buds_Air Neo True Wireless Headphone,Rs. 693,-65%,"Rs. 1,999",Free Shipping,(10)
